# **数据规整：聚合、合并和重塑**
---

## **层次化索引**
---

In [4]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                        index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]]
                        )

data

a  1    1.430802
   2   -0.203274
   3    2.661467
b  1    1.692019
   3   -1.884071
c  1    0.701836
   2    1.992013
d  2   -0.615928
   3    1.138681
dtype: float64

> **内层都可以索引**

In [5]:
data.loc[['b', 'd'],3]

b  3   -1.884071
d  3    1.138681
dtype: float64

> **data.unstack()逆透视**

In [6]:
data.unstack()

,1,2,3
a,1.430802,-0.203274,2.661467
b,1.692019,NaN,-1.884071
c,0.701836,1.992013,NaN
d,NaN,-0.615928,1.138681


> **data.stack()透视**

In [7]:
data.unstack().stack()

a  1    1.430802
   2   -0.203274
   3    2.661467
b  1    1.692019
   3   -1.884071
c  1    0.701836
   2    1.992013
d  2   -0.615928
   3    1.138681
dtype: float64

> **DataFrame每条轴都可以有分层索引**

In [8]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                    ['Green', 'Red', 'Green']]
                    )
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [9]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
# frame.index
# frame.columns
# 注意索引名、列名、行名、字段名/列名/列标签是不一样的东西
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [10]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### **重排与分级排序**

> **df.swaplevel(x,y)交换排序位置**

In [11]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [12]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

> **sort_index(level=0级别）**

In [13]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### **根据级别汇总统计**

In [14]:
frame.sum(level='key2')

C:\Users\mjl\AppData\Local\Temp\ipykernel_8648\2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [15]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
frame.sum(level='color', axis=1)

C:\Users\mjl\AppData\Local\Temp\ipykernel_8648\4133796543.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color', axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### **使用DataFrame的列进行索引**

> **set_index()**

In [17]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two',
                     'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [18]:
frame2 = frame.set_index(['c','d'],drop=False) # 保留相关列
frame2

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

## **合并数据集**  
---
> - pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来。SQL或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的join操作。  
> - pandas.concat可以沿着一条轴将多个对象堆叠到一起。  
> - 实例方法combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值。

### **类似数据库的合并**

> **pd.merge(df1,df2,on='key')**

In [19]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [20]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [21]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [22]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [23]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [24]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                        'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [25]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [26]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


> **连接方式**  
> **inner ：内连接**  
> **left : 左连接**  
> **right :右连接**  
> **Outer : 外连接**

In [27]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [28]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                     'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [29]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


> **多键合并on=['key1', 'key2']传入一个由列名组成的列表即可**

In [30]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                         'key2': ['one', 'one', 'one', 'two'],
                          'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [31]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


> **suffixes('_left', '_right')指定重复项的列明**

In [32]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


> merge参数
![](img)

### **索引上的合并**

In [33]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                     'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
pd.merge(left1, right1, left_on='key', right_index=True) # 右边使用索引作为合并的字段

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


> **merge默认取交集，可加how='outer'取并集**

In [34]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


> **多层次的索引，多键合并**

In [35]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                                'Nevada', 'Nevada'],
                        'key2': [2000, 2001, 2002, 2001, 2002],
                         'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                        index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                               'Ohio', 'Ohio'],
                               [2001, 2000, 2000, 2000, 2001, 2002]],
                       columns=['event1', 'event2'])

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [36]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
          right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


> **直接使用索引轴合并**

In [37]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                        index=['a', 'c', 'e'],
                       columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### **Join方法**

In [38]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### **concat()轴向连接**

In [39]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [40]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [41]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [42]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


> **Join='inner'取交集**

In [43]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [ ]:
pd.concat([s1, s4], axis=1, join_axes=['a', 'c', 'b', 'e'])

> **合并中还可以创建层次化索引，使用keys**

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one','two', 'three'])
result

In [ ]:
result.unstack() # 逆透视一下

In [ ]:
pd.concat([s1, s2, s3], axis=1, keys=['one','two', 'three'])

### **同样的逻辑也适用于DataFrame对象**

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                    columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                     columns=['three', 'four'])
df1                     

In [ ]:
df2

In [ ]:
pd.concat([df1,df2],axis=1,keys=['level1', 'level2'])

In [ ]:
# 如果传入的值的字典，那字典的键会被当作keys
pd.concat({'level1': df1, 'level2': df2}, axis=1) 

In [ ]:
# 设置索引层次水平
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
       names=['upper', 'lower'])

> ![](img/concat%E5%8F%82%E6%95%B0.jpg)

### **合并重叠数据combine_first打补丁**

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
               index=['f', 'e', 'd', 'c', 'b', 'a'])
a

In [ ]:
import numpy as np
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

In [ ]:
b[:-2].combine_first(a[2:])

## **重塑和轴向旋转**
---

### **重塑层次化索引**
> **df.stack()透视和df.unstack()逆透视**

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                      index=pd.Index(['Ohio','Colorado'], name='state'),
                     columns=pd.Index(['one', 'two', 'three'],
                       name='number'))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

###  **将“长格式”旋转为“宽格式”**

In [ ]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

data = data.reindex(columns=columns)

data.index = periods.to_timestamp('D', 'end')

ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
data # date和item通常就是主键

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]


In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

> **_注意，pivot其实就是用set_index创建层次化索引，再用unstack重塑：_**

### **将“宽格式”旋转为“长格式”**

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                       'A': [1, 2, 3],
                       'B': [4, 5, 6],
                      'C': [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, ['key'])
melted

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

In [ ]:
reshaped.reset_index() 